In [1]:
import simpy
import random
import numpy as np

In [2]:
seed = 978
random.seed(seed)

# Arrival mean                              --EXPONENTIAL
interarrival_mean = 6 

# Operator 1 mean and std for service time  --LOGNORMAL
m = 12 
s = 6
M = np.log(m**2/np.sqrt(m**2+s**2))
S = np.log((m**2+s**2)/m**2)

# Operator 2 service time range             --UNIFORM
service_range = [1,7]

# Voice recognition mean -                  --EXPONENTIAL
router_mean = 5

In [3]:
router_customer = []
opr_1_break_times = []
opr_2_break_times = []
shift_started = True
end = False

In [4]:
def service(env, opr):
    if opr==operator1:
        yield env.timeout(random.lognormvariate(M,S))
        
    elif opr==operator2:
        yield env.timeout(random.uniform(*service_range))

In [5]:
class Customer(object):
    def __init__(self, name, env):
        self.env = env
        self.name = name
        self.arrival_t = self.env.now
        self.action = env.process(self.call())
        
    
    def call(self):
        print('%s initiated a call at %g' % (self.name, self.env.now))
        # Voice recognition and routing
        router_customer.append(self.name)
        yield env.timeout(random.expovariate(1/router_mean))
        if random.uniform(0,1) < .3:
            self.operator = operator1
        else:
            self.operator = operator2
            
        router_customer.remove(self.name)
            
        # Voice recognition failure
        if random.uniform(0,1) < .1:
            print('%s hangs up the call at %g' % (self.name, self.env.now))
            return 
        
        print('%s is rooted to operator %d at %g' % (self.name,(self.operator==operator2)+1, self.env.now))
        
        with self.operator.request(0) as req:
        
            result = yield req | env.timeout(10)
            # Reneging for 10 mins
            if req not in result:
                print('%s is left the operator %d after 10 minutes at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
                return 
            print('%s is assigned to the operator %d at %g' % (self.name,
                                                               (self.operator==operator2)+1,
                                                               self.env.now))
            yield self.env.process(service(env,self.operator))
            print('%s is done with operator %d at %g' % (self.name,
                                                        (self.operator==operator2)+1,
                                                        self.env.now))

In [6]:
def is_available(opr):
    return opr.count == 0

In [7]:
#Determining breaks
def break_number():
    return np.random.poisson(8, size=2)

In [8]:
def insertionSort(operator_list):
    for i in range(1,len(operator_list)):
        val = operator_list[i]
        pos = i
        while pos>0 and operator_list[pos-1]>val:
            operator_list[pos]=operator_list[pos-1]
            pos = pos-1
        operator_list[pos]=val
    print(operator_list)

In [9]:
#Break request times and sorting
def break_time(opr_1_break, opr_2_break):
    for k in range(opr_1_break):
        opr_1_break_times.append(random.uniform(0,480))
    for k in range(opr_2_break):
        opr_2_break_times.append(random.uniform(0,480))
    insertionSort(opr_1_break_times)
    insertionSort(opr_2_break_times)

In [10]:
def shift(env):
    shift_started = False
    if not end:
        [opr_1_break, opr_2_break] = break_number()
        print(opr_1_break)
        print(opr_2_break)
        break_time(opr_1_break, opr_2_break)
        #opr_2_break_times.insert(0,44)
        #opr_2_break_times.insert(1,46) These are for test.
        #print(opr_2_break_times)
        for i in opr_1_break_times:
            env.process(take_a_break(env, i, operator1, shift_started))
        for i in opr_2_break_times:
            env.process(take_a_break(env, i, operator2, shift_started))
        yield env.timeout(480)
        shift_started = True
        env.process(shift(env))
        print("the time %g" % env.now)
        opr_1_break_times.clear()
        opr_2_break_times.clear()

In [11]:
def take_a_break(env, number, opr, shift_started):
    yield env.timeout(number)
    with opr.request(1) as brk:
        yield brk
        if not shift_started:
            print("break started for operator %d at %g" % ((opr == operator2)+1 , env.now))
            yield env.timeout(3)

In [12]:
def customer_generator(env):
    global end
    start = True
    for i in range(1000):
        
        #Every 480 a new shift starts.
        if start:
            env.process(shift(env))
            start = False
            
        yield env.timeout(random.expovariate(1/interarrival_mean))
        
        # Voice recognition system limit
        if len(router_customer) < 100:
            customer = Customer('Customer %s' %(i+1), env)
    end = True

In [13]:
env = simpy.Environment()
operator1 = simpy.PriorityResource(env, capacity = 1)
operator2 = simpy.PriorityResource(env, capacity = 1)
env.process(customer_generator(env))
env.run() 

8
10
[44.63307524333047, 74.39519318760645, 209.12059240981336, 255.9664319411964, 292.4094909717117, 385.08427703724, 387.8161389706391, 397.1018255104171]
[34.07604646614267, 90.0478027519914, 126.55138170720205, 174.5488473525917, 226.6161423659573, 349.81705835821435, 393.61533834425836, 414.16006451794135, 434.4019150966053, 467.5377281534712]
Customer 1 initiated a call at 26.5136
break started for operator 2 at 34.076
Customer 2 initiated a call at 34.9192
Customer 1 is rooted to operator 1 at 35.9632
Customer 1 is assigned to the operator 1 at 35.9632
Customer 2 is rooted to operator 2 at 37.3759
Customer 2 is assigned to the operator 2 at 37.3759
Customer 2 is done with operator 2 at 43.0204
Customer 1 is done with operator 1 at 46.119
break started for operator 1 at 46.119
Customer 3 initiated a call at 46.5973
Customer 3 is rooted to operator 2 at 46.7038
Customer 3 is assigned to the operator 2 at 46.7038
Customer 4 initiated a call at 51.3862
Customer 3 is done with operat

Customer 350 is assigned to the operator 2 at 2290.61
Customer 352 initiated a call at 2290.71
Customer 352 is rooted to operator 2 at 2290.8
Customer 348 is done with operator 1 at 2290.86
Customer 350 is done with operator 2 at 2294.91
Customer 352 is assigned to the operator 2 at 2294.91
Customer 353 initiated a call at 2295.19
Customer 354 initiated a call at 2297.63
Customer 354 is rooted to operator 2 at 2298.75
Customer 352 is done with operator 2 at 2300.45
Customer 354 is assigned to the operator 2 at 2300.45
Customer 353 is rooted to operator 1 at 2300.98
Customer 353 is assigned to the operator 1 at 2300.98
Customer 354 is done with operator 2 at 2301.75
Customer 355 initiated a call at 2308.05
Customer 355 is rooted to operator 1 at 2310.27
Customer 353 is done with operator 1 at 2310.88
Customer 355 is assigned to the operator 1 at 2310.88
Customer 356 initiated a call at 2314.32
Customer 357 initiated a call at 2323.15
Customer 357 is rooted to operator 2 at 2323.28
Custo

Customer 565 is rooted to operator 1 at 3651.98
Customer 565 is assigned to the operator 1 at 3651.98
Customer 564 is done with operator 2 at 3653.03
break started for operator 2 at 3653.03
Customer 567 initiated a call at 3655.35
Customer 567 is rooted to operator 1 at 3655.56
Customer 566 is rooted to operator 1 at 3655.91
Customer 568 initiated a call at 3657.87
break started for operator 2 at 3658.25
Customer 565 is done with operator 1 at 3659.35
Customer 567 is assigned to the operator 1 at 3659.35
Customer 569 initiated a call at 3659.96
Customer 570 initiated a call at 3661.12
Customer 568 is rooted to operator 2 at 3662.16
Customer 568 is assigned to the operator 2 at 3662.16
Customer 566 is left the operator 1 after 10 minutes at 3665.91
Customer 569 is rooted to operator 1 at 3666.46
Customer 570 is rooted to operator 2 at 3667.16
Customer 568 is done with operator 2 at 3667.69
Customer 570 is assigned to the operator 2 at 3667.69
Customer 567 is done with operator 1 at 3668

Customer 962 is left the operator 2 after 10 minutes at 6085.88
Customer 960 is done with operator 1 at 6086.5
break started for operator 1 at 6086.5
Customer 963 is left the operator 2 after 10 minutes at 6086.55
Customer 961 is done with operator 2 at 6087.98
Customer 958 is assigned to the operator 2 at 6087.98
Customer 965 initiated a call at 6088.83
Customer 965 is rooted to operator 2 at 6091.42
Customer 958 is done with operator 2 at 6092.94
Customer 965 is assigned to the operator 2 at 6092.94
Customer 965 is done with operator 2 at 6099.4
Customer 966 initiated a call at 6100.05
Customer 966 is rooted to operator 2 at 6102.98
Customer 966 is assigned to the operator 2 at 6102.98
Customer 967 initiated a call at 6108.5
Customer 967 is rooted to operator 1 at 6108.95
Customer 967 is assigned to the operator 1 at 6108.95
Customer 966 is done with operator 2 at 6109.18
Customer 967 is done with operator 1 at 6120.72
Customer 968 initiated a call at 6124.82
Customer 968 is rooted t